In [1]:
import torch 
import torch.nn as nn
import time 
from time import sleep
import datetime
import os 
import numpy as np 
import torch.optim as optim
import matplotlib.pyplot as plt 
from torch.utils.data import DataLoader 
from sklearn.model_selection import train_test_split
import pandas as pd
import TrainHelper as th
import CustomModels as cm
import CustomImageDataset as cid
import LossFunction as lf 
import sys
import glob
sys.path.append(os.path.join('..','segment-utils'))
import VideoAnnotationHelper as vah

device = ("cuda" if torch.cuda.is_available() else "cpu")

C:\Users\bajajp\AppData\Local\anaconda3\envs\Xmen\lib\site-packages\torchvision\datapoints\__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
C:\Users\bajajp\AppData\Local\anaconda3\envs\Xmen\lib\site-packages\torchvision\transforms\v2\__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please

In [2]:
# If you are training finetune 
fineTune = True  
model_path = "//cgmqnap.clearguide.local/data/Needles/Lumena/models/shaft_segmentation/model_mid_cyc_3_ep_200_bs_128_lr_0.0001_20240111_T180718finetuned.pt"

transferLearning = False   
transfer_model_path = "//cgmqnap.clearguide.local/data/Needles/Lumena/models/shaft_segmentation_models/model_cyc_3_ep_1000_bs_16_lr_1e-05_20230929_T060507.pt"

batch_size = 128
epochs = 200
cycles = 3

encoder_lr = 1e-5
decoder_lr = 1e-5

lossFun = lf.IoULoss()

stop_encoder_requires_grad = False
stop_decoder_requires_grad = False

parent_dir = "C:/Lumena"
train_data_folder = "train_data"
csv_filename = ["cropped_256x256_original.csv", "cropped_256x256.csv"]
img_dir = ["images_cropped_256x256_original", "images_cropped_256x256"]
mask_dir = ["masks_cropped_256x256_original","masks_cropped_256x256"]

observe_path="D:/test_dir"

In [3]:
#### Outlier ####

sub_folders = ["EL", "PC2", "PR", "EL_2", "CNMC", "CNMC.2023.11.07", "SS", "Bob"]
images_path_outliers = []
labels_path_outliers = []


for ele in sub_folders: 
    
    i_paths = glob.glob(parent_dir + "/" + train_data_folder + "/" + ele + "/images_outliers_cropped_256x256/*.png")
    m_paths = glob.glob(parent_dir + "/" + train_data_folder + "/" + ele + "/masks_outliers_cropped_256x256/*.png")
    
    images_path_outliers += i_paths 
    labels_path_outliers += m_paths 
    
    
# _, X1, _, y1 = train_test_split(images_path_outliers, labels_path_outliers, test_size = 0.3)

X1 = images_path_outliers
y1 = labels_path_outliers
#### End Outlier ####

#### Main Folders ####


images_path1 = []
labels1 = []

for k in range(len(csv_filename)):
    dfs = []
    for ele in sub_folders:
        df = pd.read_csv(parent_dir + "/" + train_data_folder + "/" + ele +"/" + csv_filename[k])
        dfs.append(df)

    for i in range(len(dfs)): 
        for name in dfs[i]["Images"]: 
            if name[3]:
                label_ele = [] 
                images_path1.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + img_dir[k] + "/" + name)
                labels1.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + mask_dir[k] + "/" + name)

#_, X2, _, y2 = train_test_split(images_path1, labels1, test_size = 0.3)
X2 = images_path1
y2 = labels1

#### End Main Folders ####

#### FineTune Folders ####

sub_folders = ["PB", "PF"]            

images_path2 = []
labels2 = []

for k in range(len(csv_filename)):
    dfs = []
    for ele in sub_folders:
        df = pd.read_csv(parent_dir + "/" + train_data_folder + "/" + ele +"/" + csv_filename[k])
        dfs.append(df)

    for i in range(len(dfs)): 
        for name in dfs[i]["Images"]: 
            if name[3]:
                label_ele = [] 
                images_path2.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + img_dir[k] + "/" + name)
                labels2.append(parent_dir + "/" + train_data_folder + "/" + sub_folders[i] + "/" + mask_dir[k] + "/" + name)

# _, X3, _, y3 = train_test_split(images_path2, labels2, test_size = 0.0)
X3 = images_path2
y3 = labels2

#### End FineTune Folders ####

images_path = X1 + X2 + X3
labels = y1 + y2 + y3

X_train, X_val, y_train, y_val = train_test_split(images_path, labels, test_size = 0.2)



train_dataset = cid.CustomImageGrayMaskDataset_dataAugmentation(X_train, y_train, ifgray = False, ifTrain = True)
val_dataset = cid.CustomImageGrayMaskDataset_dataAugmentation(X_val, y_val, ifgray = False, ifTrain = True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size = batch_size, shuffle = True,num_workers=4,pin_memory = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size = batch_size, shuffle = False,num_workers=4,pin_memory = True)

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Lumena/train_data/PC2/cropped_256x256_original.csv'

In [5]:
if fineTune: 
    print("Model loaded for finetune")
    model = torch.jit.load(model_path)
else: 
    model = cm.ConvAutoEncoder().to(device)

In [6]:
if transferLearning: 
    print("Old model weights transfered")
    old_model = torch.jit.load(transfer_model_path)
    model.load_state_dict(old_model.state_dict(), strict=False)

In [7]:
i = 0
for name, param in model.named_parameters():
    if param.requires_grad == True:
        print("NO.", i, name)
        i += 1

NO. 0 conv1.weight
NO. 1 conv1.bias
NO. 2 conv2.weight
NO. 3 conv2.bias
NO. 4 conv3.weight
NO. 5 conv3.bias
NO. 6 conv4.weight
NO. 7 conv4.bias
NO. 8 conv5.weight
NO. 9 conv5.bias
NO. 10 t_conv1.weight
NO. 11 t_conv1.bias
NO. 12 t_conv2.weight
NO. 13 t_conv2.bias
NO. 14 t_conv3.weight
NO. 15 t_conv3.bias
NO. 16 t_conv4.weight
NO. 17 t_conv4.bias
NO. 18 conv6.weight
NO. 19 conv6.bias
NO. 20 b1.weight
NO. 21 b1.bias
NO. 22 b2.weight
NO. 23 b2.bias
NO. 24 b3.weight
NO. 25 b3.bias
NO. 26 b4.weight
NO. 27 b4.bias
NO. 28 b5.weight
NO. 29 b5.bias
NO. 30 b6.weight
NO. 31 b6.bias
NO. 32 b7.weight
NO. 33 b7.bias
NO. 34 b8.weight
NO. 35 b8.bias
NO. 36 b9.weight
NO. 37 b9.bias


In [8]:
# i = 0 
# encoder_list = []
# decoder_list = []

# for param in model.parameters(): 
#     if i < 10: 
#         if stop_encoder_requires_grad:
#             param.requires_grad = False 
#         encoder_list.append(param)
#     elif i < 20: 
#         if stop_decoder_requires_grad: 
#             param.requires_grad = False 
#         decoder_list.append(param)
        
#     i += 1
    
    
# for name, param in model.named_parameters():
#     if param.requires_grad == True:
#         print(name)

In [9]:
model_name = "model" 
model_name += "_cyc_" + str(cycles) + "_ep_" + str(epochs) + "_bs_" + str(batch_size)
model_name += "_lr_" + str(encoder_lr)
model_folder = model_name +  "_{:%Y%m%d_T%H%M%S}".format(datetime.datetime.now())

vah.createFolder(observe_path + "/"  + model_folder)

Created folder 'D:/test_dir/model_cyc_3_ep_200_bs_128_lr_0.001_20240112_T094326'.


In [10]:
encoder_lr

0.001

In [11]:
for j in range(cycles):
        
    print("Training Cycle: ", j+1)

#     optimizer = torch.optim.Adam([{"params":encoder_list, "lr":encoder_lr},
#                                   {"params":decoder_list, "lr":decoder_lr}
#                                  ])

    optimizer = torch.optim.Adam(model.parameters(), lr = encoder_lr)
    
    early_stopping = th.EarlyStopping(patience = 20, min_delta = 0.01)

    for epoch in range(epochs): 
        train_loss = 0.0
        val_loss = 0.0

        start_time = time.time() 
        
        model.train()
        for data in train_loader: 

             
            images, labels = data 
            images = images.to(device)
            labels= labels.to(device)
            optimizer.zero_grad()
            output = model(images)

            loss = lossFun.forward(output, labels)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        model.eval() 
        for data in val_loader: 

            
            images, labels = data
            images = images.to(device)
            labels= labels.to(device)
            with torch.no_grad(): 
                outputs = model(images)
                loss = lossFun.forward(outputs,labels)

            val_loss += loss.item()

        train_loss = train_loss/len(train_loader)    
        val_loss = val_loss/len(val_loader)

        print('Epoch: {}  \tTraining Loss: {:.6f} \t Vali Loss: {:.6f} \t Exe Time: {:.6f}sec'.format(
        epoch, 
        train_loss, 
        val_loss,
        time.time() - start_time))

        stop = early_stopping.early_stop(val_loss, model)
        if stop: 
            model = early_stopping.get_model() 
            print("Early Stopping")
            encoder_lr /= 10
            decoder_lr /= 10
            break 

        if epoch % 10 == 0: 

            model_script = torch.jit.script(model)
            temp_name = model_name + "_{:%Y%m%d_T%H%M%S}".format(datetime.datetime.now()) + ".pt"
            model_script.save(observe_path + "/"  + model_folder + "/" + temp_name)
            print("Temprary model saved: ", temp_name)

Training Cycle:  1
Epoch: 0  	Training Loss: 0.993195 	 Vali Loss: 0.994625 	 Exe Time: 16.332732sec
Temprary model saved:  model_cyc_3_ep_200_bs_128_lr_0.001_20240112_T094347.pt
Epoch: 1  	Training Loss: 0.983302 	 Vali Loss: 0.989861 	 Exe Time: 15.269667sec
Epoch: 2  	Training Loss: 0.934533 	 Vali Loss: 0.919295 	 Exe Time: 15.280990sec
Epoch: 3  	Training Loss: 0.820889 	 Vali Loss: 0.830957 	 Exe Time: 15.189441sec
Epoch: 4  	Training Loss: 0.743906 	 Vali Loss: 0.761357 	 Exe Time: 15.579628sec
Epoch: 5  	Training Loss: 0.707007 	 Vali Loss: 0.691431 	 Exe Time: 15.377470sec
Epoch: 6  	Training Loss: 0.681825 	 Vali Loss: 0.689226 	 Exe Time: 15.192897sec
Epoch: 7  	Training Loss: 0.666491 	 Vali Loss: 0.743725 	 Exe Time: 15.368881sec
Epoch: 8  	Training Loss: 0.656701 	 Vali Loss: 0.697139 	 Exe Time: 14.637329sec
Epoch: 9  	Training Loss: 0.656763 	 Vali Loss: 0.714004 	 Exe Time: 15.010188sec
Epoch: 10  	Training Loss: 0.674715 	 Vali Loss: 0.716046 	 Exe Time: 14.696806sec


In [12]:
model_script = torch.jit.script(model)
final_model = model_name + "_{:%Y%m%d_T%H%M%S}".format(datetime.datetime.now())
if fineTune: 
    final_model += "finetuned.pt"
else: 
    final_model += ".pt"

model_script.save("//cgmqnap.clearguide.local/data/Needles/Lumena" + "/models/shaft_segmentation/" + final_model)
print("Saved trained model: ", final_model)

Saved trained model:  model_cyc_3_ep_200_bs_128_lr_0.001_20240112_T103411.pt
